In [1]:
import pandas as pd
import numpy as np
import os
import cv2
#import tensorflow as tf
import glob
import h5py
import idx2numpy as idx
import warnings
import re
from PIL import Image
from tqdm import tqdm

In [2]:
warnings.filterwarnings('ignore')

#входное разрешение модели
IMAGE_HEIGHT = 28 # Height of the input images
IMAGE_WIDTH = 28 #Width of the input images
# WORK_DIR = r'D:\work\\test_comp_vision\datasets\!_lines_w25_parsed_full' #путь к папке с файлами
WORK_DIR = r'D:/work/test_comp_vision/datasets/' # путь к рабочей папке
DATASET_DIR = r'!_lines_w25_parsed_norm'
#DATASET_IDX_IMG = r'D:\work\test_comp_vision\datasets\!_lines_w25_dataset_images_100k_upper2.idx'
#DATASET_IDX_CLS = r'D:\work\test_comp_vision\datasets\!_lines_w25_dataset_classes_100k_upper2.idx'
# DATASET_SIZE = 100000      # какую часть изображений используем для создания датасета. 0 - если нужны все данные
DATASET_TRAIN_SIZE = 125000
DATASET_VALID_SIZE = 25000
DATASET_TEST_SIZE = DATASET_VALID_SIZE
# LABELS = '0123456789АБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯабвгдеёжзийклмнопрстуфхцчшщъыьэюя'
LABELS = 'АБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯ'   # for only uppercase letters without '0' and '3' numbers
EXPORT_NAME_CSV = '!img_classes_upper2.csv'

LAST_PIC = r'10069_b'

In [3]:
"""
Структура папок:
base_folder
    image_0
        image_0_0.jpg
        image_0_0.txt
        image_0_1.jpg
        image_0_1.txt
        ...
    image_1
        image_1_0.jpg
        image_1_0.txt
        image_1_1.jpg
        image_1_1.txt
        ...
        
"""


def labels_to_int(labels=LABELS) -> dict:
    # Преобразуем букву в числвой код для создания классов в модели
    # TODO - можно вынести этот функционал в препроцессинговый слой модели - layers.StringLookup(output_mode="one_hot")
    label_nums = {}
    for i, lab in enumerate(labels):
        label_nums[lab] = i
    print(label_nums)
    return label_nums


labels_comparison = labels_to_int()     # получаем сопоставление символа к коду из датасета
labels_symbol = list(labels_comparison.keys())     # и получаем отдельно список по символам и кодам
labels_class = list(labels_comparison.values())


# TODO заменить работу с индексами над путями к файлам на нормальную обработку разрешения файлов
def get_dataframe(path):
    # pattern = r'/^(.+)(\.[^ .]+)?$/'      # для удаления разрешения из пути к файлу
    
    symbol_classes = []
    for folder in tqdm(os.listdir(path)):
        # пропускаем папки с буквой 'а', т.к. там версия текста в нижнем регистре
        # if folder[:-1] == 'a' : continue     # В целом эту проверку можно не делать, т.к. в любом случаем пропускаем буквы не из списка LABEL
        # TODO добавить ограничение на количество итераций здесь, чтобы не создавать DataSet больше нужного
        for img_file in glob.glob(f'{path}/{folder}/*.jpg'):
            # TODO заменить на pathlib.stem
            txt_file =  f"{img_file[:-3]}txt"    # меняем разрешение для открытия текста [:-3]
            # print(f'IMG_file: {img_file}\nTXT_file: {txt_file}')
            with open(txt_file, 'r') as f:
                try:
                    symbol_class = f.read()
                except Exception as e:
                    print(txt_file)
                    continue
                # symbol_class = f.read().upper()      # читаем обозначение буквы и приводим ее к верхнему регистру
                if symbol_class not in LABELS: continue      # пропускаем все символы, не входящие в искомое множество
                
            # TODO - проверить без этого преобразования но с использоованием в дальнейшем строки с ведущей r'string'
            # или же переписать весь код на использование Pathlib вместо строк с адресом в win-формате
            value = (img_file.replace('\\', '/'), labels_comparison[symbol_class])  # меняем системный флеш Винды на тот, что понимает Python
            symbol_classes.append(value)
    column_names = ['image_path', 'class_id']
    return pd.DataFrame(symbol_classes, columns=column_names)

{'А': 0, 'Б': 1, 'В': 2, 'Г': 3, 'Д': 4, 'Е': 5, 'Ё': 6, 'Ж': 7, 'З': 8, 'И': 9, 'Й': 10, 'К': 11, 'Л': 12, 'М': 13, 'Н': 14, 'О': 15, 'П': 16, 'Р': 17, 'С': 18, 'Т': 19, 'У': 20, 'Ф': 21, 'Х': 22, 'Ц': 23, 'Ч': 24, 'Ш': 25, 'Щ': 26, 'Ъ': 27, 'Ы': 28, 'Ь': 29, 'Э': 30, 'Ю': 31, 'Я': 32}


In [4]:
# Code block depricated
# Creating DataFrame with path to image and class in symbol format
"""
base_path = WORK_DIR + DATASET_DIR #путь к основной папке
df_text = pd.DataFrame()

if DATASET_SIZE > 0:      # если ограничение задано, то используем его при выборке данных
    df_text = df_text.append(get_dataframe(base_path)[:DATASET_SIZE])
else:
    df_text = df_text.append(get_dataframe(base_path))
df_text
"""

'\nbase_path = WORK_DIR + DATASET_DIR #путь к основной папке\ndf_text = pd.DataFrame()\n\nif DATASET_SIZE > 0:      # если ограничение задано, то используем его при выборке данных\n    df_text = df_text.append(get_dataframe(base_path)[:DATASET_SIZE])\nelse:\n    df_text = df_text.append(get_dataframe(base_path))\ndf_text\n'

In [5]:
#Создаем DataFrame в котором хранится путь к фалу/изображению и его класс
base_path = WORK_DIR + DATASET_DIR #путь к основной папке
df_main = get_dataframe(base_path) # сюда получаем весь датасет с числовым классом вместо буквы

valid_slice = DATASET_TRAIN_SIZE + DATASET_VALID_SIZE # ограничение для вырезки датасета валидации
test_slice = valid_slice + DATASET_TEST_SIZE          # ограничение для вырезки датасета тестирования
# test_slice = min(df_main.shape[0], test_slice)      # можно добавить эту проверку

df_train = df_main[:DATASET_TRAIN_SIZE]
df_valid = df_main[DATASET_TRAIN_SIZE:valid_slice]
df_test = df_main[valid_slice:test_slice]

print(df_main.shape)
print(df_train.shape)
print(df_valid.shape)
print(df_test.shape)

# Распределяем даные на 3 датасета
#df_train = df_train.append(get_dataframe(base_path)[:DATASET_TRAIN_SIZE])
#df_valid = df_valid.append(get_dataframe(base_path)[DATASET_TRAIN_SIZE:valid_slice])
#df_test = df_test.append(get_dataframe(base_path)[valid_slice:test_slice])
                           
#df_images = pd.DataFrame({'image_path':df_text['file_name'],
#                   'class_id':[labels_to_int[label] for label in df_text['class']]})
# df_images.to_csv(EXPORT_NAME_CSV, index=False)
df_main

100%|███████████████████████████████████████████████████████████████████████████| 23458/23458 [00:52<00:00, 445.90it/s]


(481098, 2)
(125000, 2)
(25000, 2)
(25000, 2)


,image_path,class_id
0,D:/work/test_comp_vision/datasets/!_lines_w25_...,2
1,D:/work/test_comp_vision/datasets/!_lines_w25_...,5
2,D:/work/test_comp_vision/datasets/!_lines_w25_...,5
3,D:/work/test_comp_vision/datasets/!_lines_w25_...,17
4,D:/work/test_comp_vision/datasets/!_lines_w25_...,0
...,...,...
481093,D:/work/test_comp_vision/datasets/!_lines_w25_...,28
481094,D:/work/test_comp_vision/datasets/!_lines_w25_...,2
481095,D:/work/test_comp_vision/datasets/!_lines_w25_...,15
481096,D:/work/test_comp_vision/datasets/!_lines_w25_...,4


In [6]:
# Code block depricated
# Преобразуем классы к числовым значениям
"""
labels_to_int = {}
for i, label in enumerate(LABELS):
    labels_to_int[label] = i
    # labels_to_int[i] = label
print(labels_to_int)

df_images = pd.DataFrame({'image_path':df_text['file_name'],
                   'class_id':[labels_to_int[label] for label in df_text['class']]})
# df_images.to_csv(EXPORT_NAME_CSV, index=False)
df_images
"""


"\nlabels_to_int = {}\nfor i, label in enumerate(LABELS):\n    labels_to_int[label] = i\n    # labels_to_int[i] = label\nprint(labels_to_int)\n\ndf_images = pd.DataFrame({'image_path':df_text['file_name'],\n                   'class_id':[labels_to_int[label] for label in df_text['class']]})\n# df_images.to_csv(EXPORT_NAME_CSV, index=False)\ndf_images\n"

In [6]:
row = 2134
print(df_train.iloc[row]['image_path'])
print(df_train.iloc[row]['class_id'])

D:/work/test_comp_vision/datasets/!_lines_w25_parsed_norm/10187_b/10187_b_13.jpg
9


In [7]:
print(df_train.shape)
for i, row in df_train[:5].iterrows():  
    print(row['class_id'])

(125000, 2)
2
5
5
17
0


## .IDX файл генератор

In [8]:
# Code block depricated
"""
with Image.open(r"D:\work\test_comp_vision\test_for_MindSet\pass_temp\0\0\0-0.jpg") as image:
        print(image.size)
        parsed_image = np.asarray(image)
        print(parsed_image.shape)
        parsed_image2 = np.asarray([parsed_image])
        print(parsed_image2.shape)
"""        

'\nwith Image.open(r"D:\\work\test_comp_vision\test_for_MindSet\\pass_temp\x00\x00\x00-0.jpg") as image:\n        print(image.size)\n        parsed_image = np.asarray(image)\n        print(parsed_image.shape)\n        parsed_image2 = np.asarray([parsed_image])\n        print(parsed_image2.shape)\n'

In [9]:
# преобразуем все изображения в массив np.array()
def idx_file_generator(data_frame, file_name: str):
    parsed_images = []              # создаем два файла: в одном хранятся изображения в формате np.array
    parsed_classes = []             # во втором хранятся классы
    print(f"Общее количество: {data_frame.shape[0]}")

    for i, row in tqdm(data_frame.iterrows()):

        with Image.open(row['image_path']) as image:
            parsed_images.append(np.asarray(image))  #открываем картинку по ссылке, преобразуем в массив
        #print(parsed_images[row[0]].shape)

        parsed_classes.append(row['class_id'])
        #print(parsed_classes[row[0]])

    np_images = np.asarray(parsed_images) 
    np_classes = np.asarray(parsed_classes)

    print(f"Датасет изображений к сохранению: {np_images.shape}")
    print(f"Датасет классов к сохранению: {np_classes.shape}")

    idx.convert_to_file(file_name+r'_images.idx', np_images)
    idx.convert_to_file(file_name+r'_labels.idx', np_classes)


In [10]:
idx_file_generator(data_frame=df_train, file_name=WORK_DIR+f"{DATASET_DIR}_TRAIN_{df_train.shape[0]}_UPPER_CHAR_FIXED_COLOR_NORM")
idx_file_generator(data_frame=df_valid, file_name=WORK_DIR+f"{DATASET_DIR}_VALID_{df_valid.shape[0]}_UPPER_CHAR_FIXED_COLOR_NORM")
idx_file_generator(data_frame=df_test, file_name=WORK_DIR+f"{DATASET_DIR}_TEST_{df_test.shape[0]}_UPPER_CHAR_FIXED_COLOR_NORM")

Общее количество: 125000


125000it [05:18, 392.18it/s]


Датасет изображений к сохранению: (125000, 28, 28)
Датасет классов к сохранению: (125000,)
Общее количество: 25000


25000it [01:03, 392.86it/s]


Датасет изображений к сохранению: (25000, 28, 28)
Датасет классов к сохранению: (25000,)
Общее количество: 25000


25000it [01:04, 384.92it/s]


Датасет изображений к сохранению: (25000, 28, 28)
Датасет классов к сохранению: (25000,)


In [17]:
# images = idx.convert_from_file(DATASET_IDX_IMG)
# classes = idx.convert_from_file(DATASET_IDX_CLS)

# print(f"Датасет изображений из файла: {images.shape}")
# print(f"Датасет классов из файла: {classes.shape}")

Датасет изображений из файла: (100000, 28, 28)
Датасет классов из файла: (100000,)


## .H5 файл генератор (не используем, т.к. проще было пристроить .IDX)

In [ ]:
#Создаем файл датасета для обучения детектора
#train_dataset = DataGenerator(load_images_into_memory=False, hdf5_dataset_path=None)
#images_dir = ''

#train_labels_filename = EXPORT_NAME_CSV

#train_dataset.parse_csv(images_dir=images_dir,
#                        labels_filename=train_labels_filename,
#                        input_format=['image_name', 'xmin', 'xmax', 'ymin', 'ymax', 'class_id'], # This is the order of the first six columns in the CSV file that contains the labels for your dataset. If your labels are in XML format, maybe the XML parser will be helpful, check the documentation.
#                        #input_format=['image_name', 'class_id'],
#                        include_classes='all')

#train_dataset.create_hdf5_dataset(file_path=DATASET_H5,
#                                  resize=[IMAGE_HEIGHT, IMAGE_WIDTH],
#                                  variable_image_size=True,
#                                  verbose=True)